# Les indicateurs du vivre mieux

Après la débâcle subie lors du précédent TD, vous allez maintenant devoir choisir les meilleurs indicateurs pour programmer votre algorithme d’apprentissage. Vous apprendrez également à combler les données manquantes avec une classe de *Scikit Learn*, à mettre vos données à l’échelle et à intégrer le tout dans un *pipeline*.

## À propos du jeu de données

Le fichier [*Better Life Index*](../data/better-life-index-women-2021.csv) est une extraction de plusieurs indicateurs de l’enquête *Better Life 2021*.

Notez une fois de plus que les données sélectionnées ne concernent que l’enquête menée auprès des femmes des pays de l’OCDE. Pour en savoir plus sur ces indicateurs, reportez-vous à la [documentation officielle](https://www.oecd.org/fr/wise/OCDE-indicateur-du-vivre-mieux-definitions-et-metadonnees-2021.pdf).

## Description des données

Le jeu de données est constitué de 40 observations décrites par 15 variables aléatoires :

|Variable|Signification|Type de variable|
|-|-|:-:|
|*code du pays*|Pays|qualitative|
|*country*|Pays|qualitative|
|*PS_FSAFEN*|Se sentir en sécurité quand on marche seul la nuit|quantitative discrète|
|*JE_EMPL*|Taux d’emploi|quantitative discrète|
|*JE_LTUR*|Taux de chômage de longue durée|quantitative continue|
|*SC_SNTWS*|Qualité du réseau social|quantitative discrète|
|*ES_EDUA*|Niveau d’instruction|quantitative discrète|
|*ES_STCS*|Compétences des élèves|quantitative continue|
|*ES_EDUEX*|Années de scolarité|quantitative discrète|
|*EQ_WATER*|Qualité de l’eau|quantitative discrète|
|*HS_LEB*|Espérance de vie|quantitative continue|
|*HS_SFRH*|Auto-évaluation de l’état de santé|quantitative discrète|
|*SW_LIFS*|Satisfaction à l’égard de la vie|quantitative continue|
|*PS_REPH*|Taux d’homicides|quantitative continue|
|*WL_EWLH*|Horaires de travail lourds|quantitative continue|

Chaque observation est un pays de l’OCDE (+ Russie et Afrique du Sud).

### Définition de la tâche

Sans partir d’aucune hypothèse, vous souhaitez programmer une fonction de prédiction pour l’indicateur *SW_LIFS*, à savoir la satisfaction à l’égard de la vie.

**Variable cible (*target*) :** *SW_LIFS*  
**Variables explicatives (*features*) :** ?

## Aperçu des données

Avant toute chose, chargez le fichier :

In [ ]:
# your code here

Affichez un résumé de la structure du *data frame* :

In [ ]:
# your code here

Grâce à une librairie de visualisation graphique, affichez la distribution de la variable cible afin de repérer s’il existe des données aberrantes :

In [ ]:
# your code here

## Préparation des données

### Corrélation entre les variables explicatives

Rappelez-vous, la première étape de cette partie consiste à définir les variables `target` et `features` à partir de la description du jeu de données et de la tâche à réaliser. Malheureusement, vous ne savez pas encore quelles variables explicatives choisir et il serait inutile de les employer toutes : en effet, certaines d’entre elles n’ont que peu d’incidence sur la variable cible.

Afin de repérer les corrélations entre les variables explicatives, vous allez utiliser une matrice de corrélation grâce à un outil statistique : le coefficient de corrélation de Pearson (ou *r* de Pearson). Inutile de la calculer par vous-mêmes, *Pandas* dispose d’une méthode `.corr()` pour la générer automatiquement. Affectez son résultat à une variable `correlation_matrix` :

In [ ]:
# your code here

Il est possible de n’afficher que la série *SW_LIFS* afin de repérer rapidement les variables qui impactent le plus l’indice de satisfaction à l’égard de la vie, que ce soit positivement ou négativement :

In [ ]:
correlation_matrix["SW_LIFS"]

Une autre solution consiste à afficher une *heat map* avec *Seaborn* et de repérer les couleurs les plus claires et les plus foncées pour la ligne *SW_LIFS* :

In [ ]:
# set the figure size
sns.set(rc={"figure.figsize":(10, 10)})

# display a heat map
_ = sns.heatmap(data=correlation_matrix, annot=True);

Après avoir sélectionné trois variables explicatives, vous pouvez instancier les variables `target` et `features` et constituer votre nouveau *data frame* `data` :

In [ ]:
# your code here

### Remplacer les données manquantes

Selon les variables explicatives sélectionnées, vous aurez ou n’aurez pas de données manquantes. Quoi qu’il en soit, vous pouvez exécuter les lignes qui suivent sans porter préjudice à votre programme à venir.

Vous allez ici reproduire la même stratégie qu’au cours du TD précédent, mais en optant pour un outil de *Scikit Learn* qui va remplacer les valeurs manquantes dans tout le jeu de données par une mesure de position (moyenne, médiane, mode) ou une valeur fixe définie dans un autre paramètre.

Chargez la classe `SimpleImputer` du module `sklearn.impute` puis créez-en une nouvelle instance. Par défaut, les paramètres `missing_values` et `strategy` sont respectivement fixés à `np.nan` et `mean`.

In [ ]:
# your code here

Entraînez-le sur vos données grâce à la méthode `.fit()` :

In [ ]:
# your code here

Puis appliquez-le à vos données avec la méthode `.transform()` :

In [ ]:
# your code here

Comme le résultat de la transformation est une matrice *Numpy*, il faut la réinjecter dans un *data frame* :

In [ ]:
data = pd.DataFrame(data, columns=features + [target])

Un appel de `data.info()` vous confirme qu’il n’y a plus une seule donnée manquante :

In [ ]:
data.info()

### Préparer les jeux d’entraînement et de test

Il est temps de préparer vos vecteurs `X` et `y` puis de répartir l’ensemble en jeux d’entraînement et de test :

In [ ]:
# your code here

### Mettre les données à l’échelle

Dans une étape précédente, vous avez mobilisé un transformateur (la classe `SimpleImputer`) en exécutant ses méthodes `.fit()` et `.transform()`. À présent, vous allez appeler un autre transformateur pour mettre les données à l’échelle.

Pourquoi les mettres à l’échelle ? Vous avez dû remarquer que toutes les valeurs ne respectent pas la même échelle : certaines sont en pourcentage, d’autres vont de 0 à 10 etc. Les normaliser permet de les reporter toutes à une échelle commune.

Depuis le module `sklearn.preprocessing`, importez la classe `StandardScaler` et créez-en une nouvelle instance dans une variable `scaler` et lancez l’ajustement sur les valeurs de votre *data frame* `X_train` :

In [ ]:
# your code here

Il ne vous reste plus qu’à exécuter la transformation de `X_train` et `X_test` :

In [ ]:
# your code here

## Programmation du modèle

Une étape que vous connaissez maintenant bien et qui ne vous posera guère de difficulté. Entraînez un modèle de régression linéaire sur vos données d’entraînement :

In [ ]:
# your code here

Testez l’exactitude du résultat avec les données du Canada :

In [ ]:
# data for Canada
canada = df[df["code"] == "CAN"][features].values
# scale the data
canada = scaler.transform(canada)
# prediction
model.predict(canada)

## Mise en place d’un *pipeline*

Vous le remarquez facilement, certaines opérations sont communes à toutes les tâches d’apprentissage automatique. Pour cette raison, *Scikit Learn* met à disposition un outil qui permet de souder les opérations en un *pipeline* et qui améliore nettement la vitesse d’exécution du programme.

Depuis le module `sklearn.pipeline`, importez la fonction `make_pipeline()`. Instanciez votre variable `model` avec, dans l’ordre :
1. `SimpleImputer()` ;
2. `StandardScaler()` ;
3. `LinearRegression()`.

In [ ]:
# your code here

Utilisez à présent la méthode `.fit()` pour entraîner votre modèle :

In [ ]:
# your code here

La prédiction pour le Canada reste identique :

In [ ]:
model.predict(canada)

## Évaluation du modèle

Vient à présent la dure tâche de l’évaluation du modèle. Les résultats révéleront peut-être que tout votre travail n’est pas de bonne qualité, mais ils ne doivent servir qu’à vous inciter à améliorer le modèle.

### R$^2$ score

Calculez tout d’abord le R$^2$ pour les jeux de test et d’entraînement :

In [ ]:
# your code here

### RMSE

Et maintenant, attelez-vous au calcul de la RMSE :

In [ ]:
# your code here